In [1]:
# library(devtools)
# document()
# setwd("../")
# install("itecv")
# library(itecv)
# setwd("itecv")

In [2]:
library(foreach)
library(doParallel)
library(Matching)
library(plyr)
library(tidyverse)
library(stringr)
library(magrittr)
library(caret)
library(itecv)
# detach("package:plyr", unload=TRUE)

In [3]:
loop_test = function(setup) {
    DGP = setup$DGP
    n_train = setup$n_train
    n_test = setup$n_test
    n_folds = setup$n_folds
    rep = setup$rep
    models = setup$models
    
    datas = setup_data(DGP$DGP, n_train, n_test, n_folds)
    estimates = datas %$% get_estimates(data, models, cv_index, test_index) 
    errors =  estimates %$% get_errors(cv_estimates, test_estimates, datas$aux_data) 
    true_selection_error = errors$true_selection_error %>%
        mutate(DGP=DGP$name, n_train=n_train, n_test=n_test, n_folds=n_folds, rep=rep)
    
    return(true_selection_error)
}

In [4]:
models = list(
# ranger_spec = list(method = "ranger",
#                    tune_grid = expand.grid(mtry = c(3,5),
#                                            splitrule="variance",
#                                            min.node.size=5)),
gbm_spec = list(method = "gbm",
                tune_grid = expand.grid(n.trees = seq(1,501,20), 
                                        interaction.depth=3, 
                                        shrinkage = 0.2, 
                                        n.minobsinnode=3)),
enet_spec = list(method = "enet",
                 tune_grid = expand.grid(fraction=0.5,
                                         lambda=exp(seq(-5,2,0.2))))
# nnet_spec = list(method = "nnet",
#                  tune_grid = expand.grid(size = seq(10,30,5),
#                                          decay = exp(seq(-7,0,1))))
)

simulation_params = list(DGP = powers_DGPs() %>% 
                             imap(~list(name=.y, DGP=.x)),
                         n_test = 2000,
                         n_train = 1000,
                         n_folds = c(2,5,10),
                         rep = 26:50) %>%
cross
setups = simulation_params %>%
    map(~c(.,"models"=list(models)))

In [5]:
derp = setups[[1]] %>% loop_test %>% data.frame

Warning message:
: package ‘bindrcpp’ was built under R version 3.2.5

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        5.0019             nan     0.2000    1.0730
     2        4.3742             nan     0.2000    0.5645
     3        3.9121             nan     0.2000    0.3249
     4        3.4565             nan     0.2000    0.3775
     5        3.0313             nan     0.2000    0.2939
     6        2.7298             nan     0.2000    0.1617
     7        2.4302             nan     0.2000    0.1725
     8        2.1843             nan     0.2000    0.1817
     9        2.0240             nan     0.2000    0.0991
    10        1.8681             nan     0.2000    0.1123
    20        1.0420             nan     0.2000    0.0365
    40        0.5621             nan     0.2000    0.0092
    60        0.4411             nan     0.2000   -0.0165
    80        0.3443             nan     0.2000   -0.0051
   100        0.2758             nan     0.2000   -0.0057
   120        0.2224             nan     0.2000   -0.0037
   140        

Warning message:
In bind_rows_(x, .id): binding character and factor vector, coercing into character vectorWarning message:
In bind_rows_(x, .id): binding character and factor vector, coercing into character vector

In [6]:
derp

,model,selection_method,true_hte_error,true_value,DGP,n_train,n_test,n_folds,rep
1,enet~0.5~7.38905609893065,c_benefit,0.1013806,-0.01900712,scenario_1,1000,2000,2,26
2,enet~0.5~0.0122773399030684,est_te_strata,0.04106301,-0.01900712,scenario_1,1000,2000,2,26
3,gbm~121~3~0.2~3,gain,0.6805542,-0.01900712,scenario_1,1000,2000,2,26
4,enet~0.5~0.246596963941606,match_decision,0.04953758,-0.01900712,scenario_1,1000,2000,2,26
5,gbm~1~3~0.2~3,match_mse,0.06847549,-0.01900712,scenario_1,1000,2000,2,26
6,gbm~61~3~0.2~3,prediction_error,0.501864,-0.01900712,scenario_1,1000,2000,2,26
7,enet~0.5~0.00673794699908547,qini,0.04085134,-0.01900712,scenario_1,1000,2000,2,26
8,gbm~221~3~0.2~3,random,0.8154038,-0.01900712,scenario_1,1000,2000,2,26
9,gbm~121~3~0.2~3,trans_decision,0.6805542,-0.01900712,scenario_1,1000,2000,2,26
10,gbm~1~3~0.2~3,trans_mse,0.06847549,-0.01900712,scenario_1,1000,2000,2,26


In [6]:
# results = setups %>% 
#     map(~loop_test(.)) %>%
#     bind_rows

In [7]:
cl = makeCluster(8)
registerDoParallel(cl)
 
results = foreach(setup = setups, 
        .combine = bind_rows,
        .packages = (.packages()))  %dopar%  
loop_test(setup) 

write_csv(results, "data/experimental_results_2.csv")

In [8]:
stopCluster(cl)